In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
datasets = pd.read_csv('C:/Users/HP/Datasets/smile-annotations-final.csv',names=['id','text','category'])
datasets.head()

,id,text,category
0,611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [3]:
datasets.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [4]:
datasets = datasets[~datasets['category'].str.contains('\|')]
datasets.category.value_counts()

nocode          1572
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [5]:
datasets=datasets[datasets['category']!='nocode']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
unique_categorie = datasets.category.unique()
unique_categorie

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [8]:
labelD = {}
for index,category in enumerate(unique_categorie):
    labelD[category] = index

In [9]:
labelD

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [10]:
datasets['label']=datasets.category.replace(labelD)

In [11]:
len(datasets.label.values)

1481

In [12]:
X_train,X_test,y_train,y_test = train_test_split(datasets.index.values,
                                                datasets.label.values,
                                                test_size = 0.2,
                                                random_state = 17)

In [13]:
datasets['Datatype'] = ['not_set']*datasets.shape[0]
datasets.loc[X_train, 'Datatype'] = 'train'
datasets.loc[X_test, 'Datatype'] = 'test'

In [14]:
datasets.head()

,id,text,category,label,Datatype
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,test
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
5,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [15]:
datasets.groupby(['category','label','Datatype']).count()

id  text
category     label Datatype           
angry        2     test       15    15
                   train      42    42
disgust      3     test        1     1
                   train       5     5
happy        0     test      230   230
                   train     907   907
not-relevant 1     test       36    36
                   train     178   178
sad          4     test        7     7
                   train      25    25
surprise     5     test        8     8
                   train      27    27

In [16]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [17]:
tokenzier = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

In [18]:
encoded_data_train = tokenzier.batch_encode_plus(
    datasets[datasets['Datatype']=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors='pt'
    )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [19]:
encoded_data_test = tokenzier.batch_encode_plus(
    datasets[datasets['Datatype']=='test'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors='pt'
    )

In [20]:
inputs_id_train = encoded_data_train['input_ids']
attention_mask_train = encoded_data_train['attention_mask']
labels_trian = torch.tensor(datasets[datasets.Datatype == 'train'].label.values)

In [21]:
inputs_id_test = encoded_data_test['input_ids']
attention_mask_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(datasets[datasets.Datatype == 'test'].label.values)

In [22]:
datasets_train=TensorDataset(inputs_id_train,attention_mask_train,labels_trian)
datasets_test=TensorDataset(inputs_id_test,attention_mask_test,labels_test)

In [23]:
from transformers import BertForSequenceClassification

In [24]:
model =BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels = len(labelD),
        output_attentions = False,
        output_hidden_states = False
        )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [25]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

In [26]:
batch_size = 4

In [27]:
dataloader_train = DataLoader(datasets_train, sampler = RandomSampler(datasets_train),
                             batch_size=batch_size)

In [28]:
dataloader_test = DataLoader(datasets_test, sampler = RandomSampler(datasets_test),
                             batch_size=batch_size)

In [29]:
from transformers import AdamW,get_linear_schedule_with_warmup

In [30]:
optimizer = AdamW(model.parameters(),
                 lr=1e-5,
                 eps=1e-8)

In [31]:
epochs=1
scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=len(dataloader_train)*epochs)

In [32]:
def f1_scores(pred,labels):
    pred_data=np.argmax(pred,axis=1).flatten()
    labels_data = labels.flatten()
    return f1_score(labels_data,pred_data,average='weighted')

In [33]:
def accuracy_per_labels(pred,labels):
    labels_data_dict = {v:k for k, v in labelD.items()}
    pred_data=np.argmax(pred,axis=1).flatten()
    labels_data=labels.flatten()
    
    for label in np.unique(labels_data):
        y_preds = pred_data[labels_data == label] #preditions
        y_tue= labels_data[labels_data ==label] #Actual Values
        
        print("Class Data: {}".format(labels_data_dict[label]))
        print("Accuracy Data: {}".format(len(y_preds[y_preds == label]/len(y_tue))))

In [34]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [36]:
def evaluate(dataloader_validation):
    model.eval()
    
    loss_evaluation = 0
    predictions, actual_values = [], []
    
    for batch in tqdm(dataloader_validation):
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':batch[0],
                 'attention_mask':batch[1],
                 'labels':batch[2]}
        with torch.no_grad():
            outputs = model(**inputs)
        
        loss = outputs[0]
        logits = outputs[1]
        loss_evaluation +=loss.item() 
        
        logits = logits.detach().cpu().numpy()
        labels_ids = inputs['labels'].cpu().numpy()
        
        predictions.append(logits)
        actual_values.append(labels_ids)
        
    loss_evaluation_average = loss_evaluation / len(dataloader_validation)
    
    predictions = np.concatenate(predictions,axis = 0)
    actual_values = np.concatenate(actual_values, axis=0)
    
    return loss_evaluation_average,predictions,actual_values

In [37]:
for epoch in tqdm(range(1,epochs+1)):
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train,
                       desc = "Epoch {:1d}".format(epoch),
                       leave = False,
                       disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        
        inputs ={'input_ids' : batch[0],
                 'attention_mask': batch[1],
                 'labels':batch[2]}
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(),1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'Training_loss':'{:3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), "C:/Users/HP/Datasets/Models/Bert_ft_epoch{}.model".format(epoch))
    tqdm.write("\epoch{epoch}")
    
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write("Training Loss: {}".format(loss_train_avg))
    
    loss_average, pred, actualvalues = evaluate(dataloader_train)
    validation_result = f1_scores(pred,actualvalues)
    
    tqdm.write("Validation Loss: {} ".format(loss_average))
    tqdm.write("F1 Scores: {}".format(validation_result))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/296 [00:00<?, ?it/s]

\epoch{epoch}
Training Loss: 0.8280087750707124


  0%|          | 0/296 [00:00<?, ?it/s]

Validation Loss: 0.671600377363329 
F1 Scores: 0.7144554493949637


In [40]:
from transformers import BertTokenizer, BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                       num_labels = len(labelD),
                                                       output_attentions = False,
                                                       output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
#model.load_state_dict( torch.load('C:/Users/HP/Datasets/Models/Fintuned_bert_epoch_1_gpu_trained.model',
            #map_location = torch.device('cpu')))

In [50]:
_,predictions,true_vals = evaluate(dataloader_train)

  0%|          | 0/296 [00:00<?, ?it/s]

In [52]:
accuracy_per_labels(predictions,true_vals)

Class Data: happy
Accuracy Data: 907
Class Data: not-relevant
Accuracy Data: 0
Class Data: angry
Accuracy Data: 0
Class Data: disgust
Accuracy Data: 0
Class Data: sad
Accuracy Data: 0
Class Data: surprise
Accuracy Data: 0
